# Sentiment Analysis - Naive Bayes Classifier & VADER - (NLTK)

In [359]:
import nltk
import nltk.classify.util, nltk.metrics
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews
from nltk.corpus import stopwords
import itertools
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.probability import FreqDist, ConditionalFreqDist 
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
import collections
import re

/Users/filippofrezza/anaconda/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


### Tokenizer

This function breaks down sentences into a list of words, very useful when parsing a review in a paragraph-like format. 

In [39]:
def tokens(text):
    "List all the word tokens (consecutive letters) in a text. Normalize to lowercase."
    return re.findall('[a-z]+', text.lower())

In [399]:
text = "it was not nice, cool but I believe it was just a bit expensive"

tokenized_text = tokens(text)
tokenized_text

['it',
 'was',
 'not',
 'nice',
 'cool',
 'but',
 'i',
 'believe',
 'it',
 'was',
 'just',
 'a',
 'bit',
 'expensive']

### Building lists for different emotions:

The idea is to build a list of words that distinguish the feeling that the sentence is trying to express by balancing out the number of positive and negative words that are being parsed.

In [121]:
positive_words = ['nice','awesome','cool','superb']
negative_words = ['bad','uninspired', 'expensive','disappointed','recommend others to avoid']
negations = ['not']

# Emotions Function:

This is a very simple function that checks whether it can match any of the words it reads for the given emotions it knows.

In [407]:
def emotions(sentence):
    positive_score = 0
    negaitve_score = 0
    for w in tokens(sentence):
        positive_score += positive_words.count(w)
        negaitve_score += negative_words.count(w)
    return positive_score, negative_score

emotions(text)

(2, 1)

# Naive Bayes Classifier:

In [330]:
def evaluate_classifier(featx):
    negids = movie_reviews.fileids('neg')
    posids = movie_reviews.fileids('pos')
 
    negfeats = [(featx(movie_reviews.words(fileids=[f])), 'neg') for f in negids]
    posfeats = [(featx(movie_reviews.words(fileids=[f])), 'pos') for f in posids]
 
    negcutoff = len(negfeats)*3/4
    poscutoff = len(posfeats)*3/4
    
    #Splitting the dataframe into train and test set
    
    trainfeats = negfeats[:int(negcutoff)] + posfeats[:int(poscutoff)]
    testfeats = negfeats[int(negcutoff):] + posfeats[int(poscutoff):]
 
    classifier = NaiveBayesClassifier.train(trainfeats)
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)
 
    for i, (feats, label) in enumerate(testfeats):
            refsets[label].add(i)
            observed = classifier.classify(feats)
            testsets[observed].add(i)
 
    print ('accuracy:', nltk.classify.util.accuracy(classifier, testfeats))
    classifier.show_most_informative_features()


In [331]:
def word_feats(words):
    return dict([(word, True) for word in words])
 
evaluate_classifier(word_feats)

accuracy: 0.728
Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
               insulting = True              neg : pos    =     13.0 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0
               ludicrous = True              neg : pos    =     11.8 : 1.0
                  avoids = True              pos : neg    =     11.7 : 1.0
             uninvolving = True              neg : pos    =     11.7 : 1.0
              astounding = True              pos : neg    =     10.3 : 1.0
             fascination = True              pos : neg    =     10.3 : 1.0
                 idiotic = True              neg : pos    =      9.8 : 1.0


### Stop Words:

In [333]:
stopset = set(stopwords.words('english'))
 
def stopword_filtered_word_feats(words):
    return dict([(word, True) for word in words if word not in stopset])
 
evaluate_classifier(stopword_filtered_word_feats)

accuracy: 0.724
Most Informative Features
             magnificent = True              pos : neg    =     15.0 : 1.0
             outstanding = True              pos : neg    =     13.6 : 1.0
               insulting = True              neg : pos    =     13.0 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0
               ludicrous = True              neg : pos    =     11.8 : 1.0
                  avoids = True              pos : neg    =     11.7 : 1.0
             uninvolving = True              neg : pos    =     11.7 : 1.0
              astounding = True              pos : neg    =     10.3 : 1.0
             fascination = True              pos : neg    =     10.3 : 1.0
                 idiotic = True              neg : pos    =      9.8 : 1.0


### Bigrams:

In [351]:
def best_bigram_word_feats(words, score_fn=BigramAssocMeasures.chi_sq, n=200):
    bigram_finder = BigramCollocationFinder.from_words(words)
    bigrams = bigram_finder.nbest(score_fn, n)
    d = dict([(bigram, True) for bigram in bigrams])
    d.update(best_word_feats(words))
    return d
 
print('evaluating best words + bigram chi_sq word features')
evaluate_classifier(best_bigram_word_feats)

evaluating best words + bigram chi_sq word features
accuracy: 0.854
Most Informative Features
          ('give', 'us') = True              neg : pos    =     12.3 : 1.0
              vulnerable = True              pos : neg    =     12.3 : 1.0
       ('matt', 'damon') = True              pos : neg    =     12.3 : 1.0
             uninvolving = True              neg : pos    =     11.7 : 1.0
    ('absolutely', 'no') = True              neg : pos    =     10.6 : 1.0
       ('fairy', 'tale') = True              pos : neg    =     10.3 : 1.0
       ('video', 'game') = True              neg : pos    =      9.7 : 1.0
                  symbol = True              pos : neg    =      9.7 : 1.0
     ('saving', 'grace') = True              neg : pos    =      9.7 : 1.0
    ('quite', 'frankly') = True              neg : pos    =      9.7 : 1.0


# ALTERNATIVE NAIVE BAYES CLASSIFIER : 

### Subjective and Objective documents :

The aim here is to create two type of documents, one where emotions are protrayed (subjective documents) and one where facts regarding movies are being stated (objective documents).

In [362]:
n_instances = 500
subj_docs = [(sent, 'subj') for sent in subjectivity.sents(categories='subj')[:n_instances]]
obj_docs = [(sent, 'obj') for sent in subjectivity.sents(categories='obj')[:n_instances]]

### Train and Test Sets :

In [365]:
train_subj_docs = subj_docs[:400]
test_subj_docs = subj_docs[400:]

train_obj_docs = obj_docs[:400]
test_obj_docs = obj_docs[400:]

training_docs = train_subj_docs+train_obj_docs
testing_docs = test_subj_docs+test_obj_docs

sentim_analyzer = SentimentAnalyzer()
all_words_neg = sentim_analyzer.all_words([mark_negation(doc) for doc in training_docs])

### Adding Unigrams to model's feature extraction process:

For this alternative model it appears that Unigrams work better that Bi-grams. They contribute to a significant increase in accuracy, precision, f-measure and recall. 

In [395]:
unigram_feats = sentim_analyzer.unigram_word_feats(all_words_neg, min_freq=4)

sentim_analyzer.add_feat_extractor(extract_unigram_feats, unigrams=unigram_feats)

### Results:

In [396]:
training_set = sentim_analyzer.apply_features(training_docs)
test_set = sentim_analyzer.apply_features(testing_docs)

In [397]:
trainer = NaiveBayesClassifier.train
classifier = sentim_analyzer.train(trainer, training_set)

Training classifier


In [398]:
for key,value in sorted(sentim_analyzer.evaluate(test_set).items()):
    print('{0}: {1}'.format(key, value))

Evaluating NaiveBayesClassifier results...
Accuracy: 0.905
F-measure [obj]: 0.9054726368159204
F-measure [subj]: 0.9045226130653266
Precision [obj]: 0.900990099009901
Precision [subj]: 0.9090909090909091
Recall [obj]: 0.91
Recall [subj]: 0.9


# VADER : 

This is a lexicon and rule-based tool that outputs the sentiment intensity score as well as giving the percentage of positive, netural and negative tone that emerges from the given sentence/paragraph. VADER also outputs a compound score, which basically is a normalized score for the sentiment intensity. 

In [408]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

### Creating the corpus of sentences:

#### Sentences:

In [410]:
sentences = ["VADER is smart, handsome, and funny.", # positive sentence example
"VADER is smart, handsome, and funny!", # punctuation emphasis handled correctly (sentiment intensity adjusted)
"VADER is very smart, handsome, and funny.",  # booster words handled correctly (sentiment intensity adjusted)
"VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
"VADER is VERY SMART, handsome, and FUNNY!!!",# combination of signals - VADER appropriately adjusts intensity
"VADER is VERY SMART, really handsome, and INCREDIBLY FUNNY!!!",# booster words & punctuation make this close to ceiling for score
"The book was good.",         # positive sentence
"The book was kind of good.", # qualified positive sentence is handled correctly (intensity adjusted)
"The plot was good, but the characters are uncompelling and the dialog is not great.", # mixed negation sentence
"A really bad, horrible book.",       # negative sentence with booster words
"At least it isn't a horrible book.", # negated negative sentence with contraction
":) and :D",     # emoticons handled
"",              # an empty string is correctly handled
"Today sux",     #  negative slang handled
"Today sux!",    #  negative slang with punctuation emphasis handled
"Today SUX!",    #  negative slang with capitalization emphasis
"Today kinda sux! But I'll get by, lol" # mixed sentiment example with slang and constrastive conjunction "but"
            ]

#### Paragraphs:

In [412]:
paragraph = "It was one of the worst movies I've seen, despite good reviews. \
Unbelievably bad acting!! Poor direction. VERY poor production. \
The movie was bad. Very bad movie. VERY bad movie. VERY BAD movie. VERY BAD movie!"

lines_list = tokens(paragraph) #this is exactly where the tokenizer comes in handy
sentences.extend(lines_list)

#### Tricky Sentences:

In [413]:
tricky_sentences = [
"Most automated sentiment analysis tools are shit.",
"VADER sentiment analysis is the shit.",
"Sentiment analysis has never been good.",
"Sentiment analysis with VADER has never been this good.",
"Warren Beatty has never been so entertaining.",
"I won't say that the movie is astounding and I wouldn't claim that \
the movie is too banal either.",
"I like to hate Michael Bay films, but I couldn't fault this one",
"It's one thing to watch an Uwe Boll film, but another thing entirely \
to pay for it",
"The movie was too good",
"This movie was actually neither that funny, nor super witty.",
"This movie doesn't care about cleverness, wit or any other kind of \
intelligent humor.",
"Those who find ugly meanings in beautiful things are corrupt without \
being charming.",
"There are slow and repetitive parts, BUT it has just enough spice to \
keep it interesting.",
"The script is not fantastic, but the acting is decent and the cinematography \
is EXCELLENT!",
"Roger Dodger is one of the most compelling variations on this theme.",
"Roger Dodger is one of the least compelling variations on this theme.",
"Roger Dodger is at least compelling as a variation on the theme.",
"they fall in love with the product",
"but then it breaks",
"usually around the time the 90 day warranty expires",
"the twin towers collapsed today",
"However, Mr. Carter solemnly argues, his client carried out the kidnapping \
under orders and in the ''least offensive way possible.''"
]

In [414]:
#adding tricky sentences to the corpus
sentences.extend(tricky_sentences)

# Outcome :

In [415]:
sid = SentimentIntensityAnalyzer()
for sentence in sentences:
    print(sentence)
    ss = sid.polarity_scores(sentence)
    for k in sorted(ss):
        print('{0}: {1}, '.format(k, ss[k]), end='')
        print()

VADER is smart, handsome, and funny.
compound: 0.8316, 
neg: 0.0, 
neu: 0.254, 
pos: 0.746, 
VADER is smart, handsome, and funny!
compound: 0.8439, 
neg: 0.0, 
neu: 0.248, 
pos: 0.752, 
VADER is very smart, handsome, and funny.
compound: 0.8545, 
neg: 0.0, 
neu: 0.299, 
pos: 0.701, 
VADER is VERY SMART, handsome, and FUNNY.
compound: 0.9227, 
neg: 0.0, 
neu: 0.246, 
pos: 0.754, 
VADER is VERY SMART, handsome, and FUNNY!!!
compound: 0.9342, 
neg: 0.0, 
neu: 0.233, 
pos: 0.767, 
VADER is VERY SMART, really handsome, and INCREDIBLY FUNNY!!!
compound: 0.9469, 
neg: 0.0, 
neu: 0.294, 
pos: 0.706, 
The book was good.
compound: 0.4404, 
neg: 0.0, 
neu: 0.508, 
pos: 0.492, 
The book was kind of good.
compound: 0.3832, 
neg: 0.0, 
neu: 0.657, 
pos: 0.343, 
The plot was good, but the characters are uncompelling and the dialog is not great.
compound: -0.7042, 
neg: 0.327, 
neu: 0.579, 
pos: 0.094, 
A really bad, horrible book.
compound: -0.8211, 
neg: 0.791, 
neu: 0.209, 
pos: 0.0, 
At least it i